In [1]:
#!apt update
#!apt install chromium-chromedriver
!pip install selenium

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait

import pandas as pd

In [46]:

def extract_value_from_format(input):
    value = None

    if "%" in input:
        value = float(input.replace("%", ""))
    elif "M" in input:
        value = float(input.replace("M", "")) * 1000 * 1000
    elif "B" in input:
        value = float(input.replace("B", "")) * 1000 * 1000 * 1000
    elif input.replace('.','',1).isdigit():
        value = float(input)
    elif "N/A" in input:
        pass  # value = None


    else:
        print("input not preprocessed: ", input)

    return value

In [43]:
def get_key_from_metric_label(label):
    return label.replace(" ", "_")

def add_metrics_from_site(driver, metrics, metrics_by_label):
    
    if len(driver.find_elements(By.ID, 'Col1-0-KeyStatistics-Proxy')) == 0:
        print("ERROR: no key-statistics page found! Returns empty metrics")
        return metrics + [None] * (len(metrics_by_label) - 1) # return placeholders
    
    
    metrics_body = driver.find_element(By.ID, 'Col1-0-KeyStatistics-Proxy')
    
    for metric_label in metrics_by_label:
        if metric_label == Metrics.TICKER: # tickername only for dataframe-label reasons. Maybe change logic to more sothisticated list 
            continue
            
        market_cap_element = metrics_body.find_element(By.XPATH, "//tr[contains(., '"+metric_label+"')]")
        child_elements = market_cap_element.find_elements(By.XPATH, "./*")

        assert(len(child_elements) == 2)
        
        # extract values from format
        value = extract_value_from_format(child_elements[1].text) 
        metrics.append(value)
        #metrics.append(child_elements[1].text)
        
        # TODO!
        #if get_key_from_metric_label(child_elements[0].text) != metric_label:
        #    print("Caution: modified metric-name from website " + get_key_from_metric_label(child_elements[0].text) + " differs from the used column-key: "+ metric_label )
        
        
        #metrics[get_key_from_metric_label(child_elements[0].text)] = child_elements[1].text
        #metrics.append((child_elements[0].text, child_elements[1].text))  # TODO return metric_label value (but be aware of the contains-search!)

    return metrics
    

In [38]:
# supporting class with constants from the website (keys etc.)
class Metrics(object):
    
    # Valuation Measures
    MARKET_CAP = 'Market Cap'
    MARKET_CAP_KEY = 'Market_Cap'
    ENTERPRISE_VALUE = 'Enterprise Value'
    ENTERPRISE_VALUE_KEY = 'Enterprise_Value'
    
    
    # Profitability
    PM = "Profit Margin"
    OM = "Operating Margin"
    
    # Management Effectiveness
    ROA = "Return on Assets"
    ROE = "Return on Equity"
    
    # Balance Sheet
    TC = "Total Cash (mrq)"  # CAUTION! search only by contains, not equals!
    TCPS = "Total Cash Per Share (mrq)"
    TD = "Total Debt (mrq)"  # CAUTION! search only by contains, not equals!
    TDE = "Total Debt/Equity (mrq)"
    # ...
    
    # Cash Flow Statement
    OCF = "Operating Cash Flow"
    LFCF = "Levered Free Cash Flow"
    
    # Dividiens & Splits
    FADR = "Forward Annual Dividend Rate"
    FADY = "Forward Annual Dividend Yield"
    TADR = "Trailing Annual Dividend Rate"
    TADY = "Trailing Annual Dividend Yield"
    Y5ADY = "5 Year Average Dividend Yield"
    PR = "Payout Ratio"
    # ...
    
    base = []
    TICKER = "Ticker"
    
    
    def __init__(self):
        self.base = [self.TICKER]
    
    def basics(self, addBase=False):
        if addBase:
            return self.base + [self.MARKET_CAP, self.ENTERPRISE_VALUE]
        else:
            return [self.MARKET_CAP, self.ENTERPRISE_VALUE]
    
    def diviends(self, addBase=False):
        if addBase:
            return self.base + [self.FADR, self.FADY, self.TADR, self.TADY, self.Y5ADY, self.PR]
        else:
            return [self.FADR, self.FADY, self.TADR, self.TADY, self.Y5ADY, self.PR]
        
    
    def dividends_slides(self, addBase=False):
        if addBase:
            return self.base + [self.FADY, self.TADY, self.Y5ADY, self.PR, self.PM, self.ROE, self.TC]
        else:
            return [self.FADY, self.TADY, self.Y5ADY, self.PR, self.PM, self.ROE, self.TC]
    
    def stability_slides(self, addBase=False):
        if addBase:
            return self.base + [self.TDE, self.OCF, self.LFCF]
        else:
            return [self.TDE, self.OCF, self.LFCF]
    
    def from_slides(self):
        return self.dividends_slides(addBase=True) + self.stability_slides()
    
    

class Tickerinfo(object):  # for additional type security
    
    def __init__(self, name, url):
        self.name = name
        self.url = url
        

In [47]:
options = webdriver.ChromeOptions()
options.add_argument('--headless') # for not displaying the graphical environment, shows virtualized browser without GUI
options.add_argument('--no-sandbox') # so that it can access machine resources, blocking sandbox processes it can access whatever
options.add_argument('--disable-dev-shm-usage')  # colab does not have enough memory
# open it, go to a website, and get results
driver = webdriver.Chrome(options=options)

url = "https://finance.yahoo.com/"
driver.get(url)

try:
    # Accept cookies by clicking the button with the specified ID
    print("accept cookies")
    iframe = driver.find_element(By.CLASS_NAME, 'con-wizard')
    accept_cookies_button = iframe.find_element(By.CLASS_NAME, 'accept-all')
    accept_cookies_button.click()
    
    
    print("call trending tickers")
    # call url with tickers:
    driver.get("https://finance.yahoo.com/trending-tickers")
    
    tab = driver.find_element(By.TAG_NAME, 'tbody')
    tickers = []
    assert(tab)
    elements = tab.find_elements(By.TAG_NAME, 'tr')
    
    print("amount of tickers: ", len(elements))
    #elements = elements[:5]  # TODO, take all tickers (only for testing)
    
    
    links = [e.find_element(By.TAG_NAME, 'a') for e in elements]

    for l in links:
        tickers.append(Tickerinfo(l.text, l.get_attribute("href")))
    
    print("getting tickers finished")
    
    required_metrics = Metrics().from_slides()
    
    df_metrics = pd.DataFrame(columns=[get_key_from_metric_label(label) for label in required_metrics])
    
    # call metric-webpage for each ticker and scrape values
    for ticker in tickers:
        #if not ticker.name == "ES=F":
        #    continue
            
        tickername = ticker.name
        print()
        print("ticker: ", tickername)
        url = "https://finance.yahoo.com/quote/"+tickername+"/key-statistics?p="+tickername
        driver.get(url)

        metrics = [tickername]
         # get metric values from website
        print("start scraping metrics for ticker from: ", driver.current_url)
        metrics = add_metrics_from_site(driver, metrics, required_metrics)

         # add metrics as new last row to df
        df_metrics.loc[len(df_metrics)] = metrics

    #print(df_metrics)

finally:
    # Close the WebDriver
    driver.quit()
    
print()
print("finished scraping")

accept cookies
call trending tickers
amount of tickers:  30
getting tickers finished

ticker:  PFE
start scraping metrics for ticker from:  https://finance.yahoo.com/quote/PFE/key-statistics?p=PFE

ticker:  PATH
start scraping metrics for ticker from:  https://finance.yahoo.com/quote/PATH/key-statistics?p=PATH


C:\Users\bened\AppData\Local\Temp\ipykernel_31836\3239735867.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_metrics.loc[len(df_metrics)] = metrics



ticker:  ULTA
start scraping metrics for ticker from:  https://finance.yahoo.com/quote/ULTA/key-statistics?p=ULTA


C:\Users\bened\AppData\Local\Temp\ipykernel_31836\3239735867.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_metrics.loc[len(df_metrics)] = metrics



ticker:  MRVL
start scraping metrics for ticker from:  https://finance.yahoo.com/quote/MRVL/key-statistics?p=MRVL

ticker:  ESTC
start scraping metrics for ticker from:  https://finance.yahoo.com/quote/ESTC/key-statistics?p=ESTC


...


ticker:  SMCI
start scraping metrics for ticker from:  https://finance.yahoo.com/quote/SMCI/key-statistics?p=SMCI


C:\Users\bened\AppData\Local\Temp\ipykernel_31836\3239735867.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_metrics.loc[len(df_metrics)] = metrics



ticker:  RIOT
start scraping metrics for ticker from:  https://finance.yahoo.com/quote/RIOT/key-statistics?p=RIOT


C:\Users\bened\AppData\Local\Temp\ipykernel_31836\3239735867.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_metrics.loc[len(df_metrics)] = metrics



finished scraping


In [35]:
#x = float('3.7v8') #.isnumeric()
x = '3,78'.replace('.','',1).isdigit()
print(x)

False


In [ ]:
# done: postprocess data within dataframe

# todo: import NASDAQ-100 tickers

# scrape dividend data for tickers:

# https://finance.yahoo.com/quote/AAPL/history?period1=1669797901&period2=1701333901&interval=capitalGain%7Cdiv%7Csplit&filter=div&frequency=1d&includeAdjustedClose=true

In [ ]:
relative_column = "Forward_Annual_Dividend_Yield"

df_metrics["Payout_Ratio"]


In [48]:
df_metrics

,Ticker,Forward_Annual_Dividend_Yield,Trailing_Annual_Dividend_Yield,5_Year_Average_Dividend_Yield,Payout_Ratio,Profit_Margin,Return_on_Equity,Total_Cash_(mrq),Total_Debt/Equity_(mrq),Operating_Cash_Flow,Levered_Free_Cash_Flow
0,PFE,5.38,5.35,3.78,89.07,15.29,11.06,4.418000e+10,65.98,1.204000e+10,-4.702500e+08
1,PATH,NaN,0.00,NaN,0.00,-12.51,-7.93,1.820000e+09,2.94,2.474400e+08,3.760400e+08
2,ULTA,NaN,0.00,NaN,0.00,11.37,62.64,1.218100e+08,103.52,1.270000e+09,6.263500e+08
3,MRVL,0.43,0.11,0.65,11.01,-10.11,-3.61,7.256000e+08,27.48,1.180000e+09,1.500000e+09
4,ESTC,NaN,0.00,NaN,0.00,-19.33,-53.43,9.570900e+08,143.65,8.318000e+07,2.662200e+08
5,BABA,NaN,0.00,NaN,0.00,14.50,11.31,5.743600e+11,16.96,2.133100e+11,1.296500e+11
6,ALT,NaN,0.00,NaN,0.00,0.00,-44.15,1.407800e+08,0.52,-7.658000e+07,-5.197000e+07
7,U,NaN,0.00,NaN,0.00,-42.31,-30.35,1.510000e+09,82.55,1.135400e+08,4.924300e+08
8,BTC-USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,MARA,NaN,0.00,NaN,0.00,-133.72,-42.13,1.012100e+08,31.94,-3.172200e+08,-2.067500e+08


In [56]:

df_metrics.to_csv('./../Dataset/data.csv')